# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split # импорт функции деления датасета
from sklearn.metrics import accuracy_score #метод оценки доли правильных ответов
from sklearn.tree import DecisionTreeClassifier #метод дерева принятия решений
from sklearn.ensemble import RandomForestClassifier # метод случайного леса
from sklearn.linear_model import LogisticRegression # метод логистической регрессии
from sklearn.dummy import DummyClassifier # модель Думми

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Предсталенные данные не имеют пропусков.<br>

Количество исследований 3214, содержат в себе количество звонков(сalls), суммарную длительность звонков в минутах (minutes), количество sms-сообщений (messages), израсходованный интернет-трафик в Мб(mb_used) и тип тарифа (is_ultra, где «Ультра» — 1, «Смарт» — 0).<br>

Представленние данные соответствуют типу и предобработаны (согласно постановке задачи).

## Разбейте данные на выборки

In [4]:
# Разделение данных на 3 выборки: обучающую, валидационную, тестовую
df_train, df_valid_test = train_test_split(df, train_size=0.7, test_size=0.3, random_state=12345)
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

In [5]:
display(len(df_train)) # обучающая
display(len(df_valid)) # валидационная
display(len(df_test)) # тестовая

2249

482

483

### Разделение выборок

#### Разделение обучающей выборки на признаки и целевой признак

In [6]:
features_train = df_train.drop(['is_ultra'],axis=1)
target_train = df_train['is_ultra']
display(features_train.head(3))# признаки
display(target_train.head(3))# целевой признак

,calls,minutes,messages,mb_used
372,100.0,626.94,43.0,22690.35
703,28.0,195.36,29.0,11179.93
2406,53.0,359.13,0.0,13758.39


372     1
703     0
2406    0
Name: is_ultra, dtype: int64

#### Разделение валидационной выборки на признаки и целевой признак

In [7]:
features_valid = df_valid.drop(['is_ultra'],axis=1)
target_valid = df_valid['is_ultra']
display(features_valid.head(3))# признаки
display(target_valid.head(3))# целевой признак

,calls,minutes,messages,mb_used
2487,67.0,439.01,20.0,24095.57
864,6.0,35.11,34.0,17625.23
1647,41.0,243.63,42.0,20579.46


2487    0
864     0
1647    0
Name: is_ultra, dtype: int64

#### Разделение тестовой выборки на признаки и целевой признак

In [8]:
features_test = df_test.drop(['is_ultra'],axis=1)
target_test = df_test['is_ultra']
display(features_test.head(3))# признаки
display(target_test.head(3))# целевой признак

,calls,minutes,messages,mb_used
2208,22.0,175.96,29.0,14303.32
1584,27.0,192.57,105.0,23746.70
2584,46.0,298.79,39.0,19376.93


2208    0
1584    1
2584    1
Name: is_ultra, dtype: int64

<b> Разделение тестовой, тренировочной и валидационной выборки на признаки(features) и целевой признак(target) прошло успешно.

## Исследуйте модели

### Обучение модели "Дерево решений"

In [9]:
for depth in range(1,7): # оптимально-показательное количество, т.к. при глубине более 6 значения преимущественно ухудшались
    model_tree=DecisionTreeClassifier(random_state=12345, max_depth=depth) # присваиваем модели имя
    model_tree.fit(features_train,target_train) # обучаем
    prediction_tree=model_tree.predict(features_valid) # проверяем
    print('max_depth:',depth,end='')
    print(' accuracy:',accuracy_score(prediction_tree,target_valid))

max_depth: 1 accuracy: 0.7510373443983402
max_depth: 2 accuracy: 0.7800829875518672
max_depth: 3 accuracy: 0.7863070539419087
max_depth: 4 accuracy: 0.7883817427385892
max_depth: 5 accuracy: 0.7614107883817427
max_depth: 6 accuracy: 0.7842323651452282


<b> По результатам обучения видим, что наилучшая глубина 4 (лучшее значение гиперпараметров) и качество модели = 0.788

### Обучение модели "Случайный лес"

In [10]:
ac=0
es=0
de=0
for estim in range(1,50):    
    model_forest = RandomForestClassifier(max_depth=10,n_estimators=estim,random_state=12345)
    model_forest.fit(features_train,target_train) # обучаем
    prediction_forest = model_forest.predict(features_valid) # проверяем
    accuracy = accuracy_score(target_valid, prediction_forest)
    if accuracy > ac:
        ac=accuracy
        es=estim
        de=depth
print("n_estimators:", es, "depth:", de, "accuracy", ac)

n_estimators: 24 depth: 6 accuracy 0.8132780082987552


<b>По результатам обучения видим, что наилучшая глубина 6 (лучшее значение гиперпараметров) и качество модели = 0.813

### Обучение модели "Логистическая регрессия"

In [11]:
model = LogisticRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print('accuracy:', accuracy)

accuracy: 0.6950207468879668


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


<b> По результатам видно, что модель не эффективна. Качество модели "Логистическая регрессия" = 0.695, что является самым низким показателем.

<b> Вывод:</b><br>
В результате исследования 3-х разных моделей получены следующие показатели: <br>
 - Качество модели "Дерево предсказания", при гиперпараметрах глубины = 4, составляет 0.788;
 - Качество модели "Случайный лес" при гиперпараметрах глубины = 6, составляет 0.813;
 - Качество модели "Логистическая регрессия": 0.695, что является самым худшим показателем.

<b>Лучшей для данной выборки оказалась модель "Случайный лес"

## Проверьте модель на тестовой выборке

<b>Тестирование данных на модели "Случайный лес"

In [12]:
model_forest = RandomForestClassifier(max_depth=10,n_estimators=24,random_state=12345)
model_forest.fit(features_train,target_train)
prediction_forest = model_forest.predict(features_test)
print(' accuracy:',accuracy_score(prediction_forest,target_test))

 accuracy: 0.8157349896480331


<b> Модель проверены на тестовой выборке. Метрики качества тестовой выборки сопоставимы с метрикам качества валидационной модели

## (бонус) Проверьте модели на адекватность

Наиболее простая модель, которая предсказывает наиболее часто встречающийся класс - модель Думми

In [13]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

0.6927523343708315

<b> Вывод </b> <br>
В результате исследования определено, что наилучшим образом для данной выборки подходит модель <b>"Случайный лес"</b>, которая показывает около <b>82%</b> качество предсказания, что на 12% больше тестовой модели Думми и достаточно для выполнения поставленных в проекте задач.